<a href="https://colab.research.google.com/github/Ch-Lokesh/tf_chat_bot/blob/master/tf_chat_bot.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Intent is important for a chat bot, so we make intent classificaion, we make tags, pattern and responses

In [4]:
#these are libraries for NLP

import nltk #natural language processing tool kit
nltk.download('punkt')  #we need to download punkt to tokanise sentences into individual words
from nltk.stem.lancaster import LancasterStemmer
stemmer = LancasterStemmer() #creating an object, stemmming reduces a words into root word(eg cooks,cooking , cooked, cook into cook)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.


In [0]:
#these are libraries for tensorflow
import tensorflow as tf
import numpy as np
import tflearn
import random
import json     #to read json file
import pickle   #to save the model


In [7]:
from google.colab import files
files.upload()

Saving intent.json to intent.json


{'intent.json': b'{\n    "intents":[\n        {"tag" : "greetings",\n        "patterns" :["Hi", "How are you", "Is anyone there?", "Hello", "Good Day"],\n        "responses" :["Hello", "hii", "Hey there", "Good to See You", "How can I help You"],\n        "context_set" : ""\n        },\n        {"tag" : "good bye",\n        "patterns" :["Bye", "See you later", "Text you later", "Good bye"],\n        "responses" : ["Bye ! Come back again", "Have a nice day", "See you later", "Nice to meet you"]},\n        \n        {"tag" : "thanks",\n        "patterns" :["Thanks", "Thank you so much", "That\'s heplful"],\n        "responses" : ["Happy to help!", "Any time!", "My Pleasure"]},\n\n        {"tag" : "location",\n        "patterns" :["where do you live ?", "what is your native place ?", "what is your home town"],\n        "responses" :["My home town is Warangal"]}\n    ]\n}'}

In [0]:
#importing chat bots intent file
with open('intent.json') as json_data:
    intents = json.load(json_data)

In [0]:
intents

In [0]:
words = []
classes = []
documents = []
ignore = ['?', '.', '!']
#loop through each sentence in the intent's patterns
for intent in intents['intents']:
    for pattern in intent['patterns']:
        #tokanise each and every word in the sentence
        w = nltk.word_tokenize(pattern)
        #adding word to word list
        words.extend(w)
        #adding word(s) to the documents
        documents.append((w, intent['tag']))
        #add tags to our classes list
        if intent['tag'] not in classes:
            classes.append(intent['tag'])

In [11]:
#perform stemming and lower each word as well as remove duplicates
words = [stemmer.stem(w.lower())  for w in words if w not in ignore]
words = sorted(list(set(words)))

#removing duplicate classes

print(len(documents), "documents")
print(len(classes), "classes", classes)
print(len(words), "unique stemmed words", words)


15 documents
4 classes ['greetings', 'good bye', 'thanks', 'location']
29 unique stemmed words ["'s", 'anyon', 'ar', 'bye', 'day', 'do', 'good', 'hello', 'hepl', 'hi', 'hom', 'how', 'is', 'lat', 'liv', 'much', 'nat', 'plac', 'see', 'so', 'text', 'thank', 'that', 'ther', 'town', 'what', 'wher', 'yo', 'you']


In [0]:
#creaing a training data i.e transforming the documents into tensers
training = []
output = []

#creating an empty array for output
output_empty = [0]*len(classes)

#creating  a training set, a bag of words for each sentence
for doc in documents:
    #initialising the bag of words
    bag = []
    
    #list of tokanised words
    pattern_words =  doc[0]
    #stemming each word
    pattern_words = [stemmer.stem(word.lower()) for word in pattern_words]
    
    #creating bag of words array
    for w in words:
        bag.append(1) if w in pattern_words else bag.append(0)
        
    #output is 1 for current tag and 0 for others
    output_row = list(output_empty)
    output_row[classes.index(doc[1])] = 1
    
    training.append([bag, output_row])
  

In [0]:
random.shuffle(training)
training = np.array(training)

#creating training lists

train_x = list(training[:, 0])    #features
train_y = list(training[:, 1])    #labels in the form assosiated tags or intent class



In [15]:
# resetting underlying graph data
import tensorflow as tf
tf.reset_default_graph()

#building a neural network
net = tflearn.input_data(shape = [None, len(train_x[0])]) #input layer a placeholder will be used a shape or a placeholder must be used
net = tflearn.fully_connected(net, 10)    #this and the below are hidden layers
net = tflearn.fully_connected(net, 10)
net = tflearn.fully_connected(net, len(train_y[0]), activation = 'softmax')   #output layer
net = tflearn.regression(net)     #regression linear reduces the loss

#defining model and setting up tensorboard
model = tflearn.DNN(net, tensorboard_dir = 'tflearn_logs')

#start training
model.fit(train_x, train_y, n_epoch = 1000, batch_size = 8, show_metric = True)
model.save('model.tflearn')

Training Step: 1999  | total loss: 0.25237 | time: 0.005s
| Adam | epoch: 1000 | loss: 0.25237 - acc: 0.9542 -- iter: 08/15
Training Step: 2000  | total loss: 0.25237 | time: 0.009s
| Adam | epoch: 1000 | loss: 0.25237 - acc: 0.9542 -- iter: 15/15
--


In [0]:
pickle.dump({'words' :words, 'classes' : classes, 'train_x' :train_x, 'train_y':train_y}, open("training_data", "wb"))

In [0]:
#restoring all the data structures
data = pickle.load(open("training_data", "rb"))
words = data['words']
classes = data['classes']
train_x = data['train_x']
train_y = data['train_y']

In [0]:
with open('intent.json') as json_data:
    intents = json.load(json_data)

In [25]:
# load the saved model
model.load( './model.tflearn' )

W0807 17:37:43.593730 139882482620288 deprecation.py:323] From /usr/local/lib/python3.6/dist-packages/tensorflow/python/training/saver.py:1276: checkpoint_exists (from tensorflow.python.training.checkpoint_management) is deprecated and will be removed in a future version.
Instructions for updating:
Use standard file APIs to check for files with this prefix.


In [0]:
def clean_up_sentence(sentence):
    #tokanize the pattern
    sentence_words = nltk.word_tokenize(sentence)
    #stemming each word
    sentence_words = [stemmer.stem(word.lower()) for word in sentence_words]
    return sentence_words

#returning bag of words array: 0 or 1 for each word in the bag that exits in sentence

def bow(sentence, words, show_details = False):
    #tokanize the pattern
    sentence_words = clean_up_sentence(sentence)
    #generating bag of words
    bag = [0]*len(words)
    
    for s in sentence_words:
        for i, w in enumerate(words):
            if w == s:
                bag[i] = 1
                if show_details:
                    print("found in bag :%s", w)
    return (np.array(bag))

In [0]:
error_threshold = 0.30
def classify(sentence):
    #generate probabilities from the model
    results = model.predict([bow(sentence, words)])[0]
    
    #filter out perdictions below a threshold 
    results = [[i, r] for i, r in enumerate(results) if r > error_threshold]
    #sorting by strength of probability
    results.sort(key = lambda x: x[1], reverse = True)
    return_list = []
    
    for r in results:
        return_list.append((classes[r[0]], r[1]))
        
    #return tuple of intent and probability
    return return_list

def response(sentence, userID = '123', show_details = False):
    results = classify(sentence)
    #if we have a classification then find the matching intent taf
    if results:
        for i in intents['intents']:
            #find a tag matching the first result
            if i['tag'] == results[0][0]:
                #a random response from the intent
                return print(random.choice(i['responses']))
        results.pop(0)

In [38]:
classify("how are you?")

[('greetings', 0.97694325)]

In [41]:
response('how are you')

Good to See You


In [42]:
response("how are you?")

Hey there


In [43]:
response("good bye chat bot")

Nice to meet you


In [44]:
response('do you accept credit cards')

Nice to meet you


In [45]:
response('bye')

Bye ! Come back again
